In [2]:
%load_ext dotenv
%dotenv

In [3]:
import boto3, json, sagemaker, time
import torch
import os
from sagemaker import get_execution_role
from transformers import AutoProcessor

/Users/jaredpangallozzi/PycharmProjects/llm_image_captioner/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
sess = boto3.Session()
sm = sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=sess)
role = get_execution_role()
client = boto3.client("sagemaker-runtime")

In [5]:
triton_image_uri = "{account_id}.dkr.ecr.{region}.amazonaws.com/sagemaker-tritonserver:21.08-py3".format(
    account_id=os.environ["AWS_ACCOUNT_ID"], region="us-east-1"
)

In [6]:
# Our LLM, Llama, was downloaded via https://github.com/facebookresearch/llama?fbclid=IwAR3DJH6IFhcRGQJGRjUFFV1aXMjGJxXqMy5O_A9Hs1aNrl3Tdm5bUIHfSrA
# since it was easier to run Meta's provided shell script than download from Huggingface. The 7billion parameter model was used

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = torch.load("../blip/pytorch_model.bin")

# load json config file for blip
# put it into BlipConfig object
# put *that* object into BlipForConditionalGeneration object
# save 

print(type(model))
bs = 128
seq_len = 128
dummy_inputs = [
    torch.randint(1000, (bs, seq_len)).to(device),
    torch.zeros(bs, seq_len, dtype=torch.int).to(device),
]
print(model.keys())
# model = model.eval()
# model.to(device)

# traced_model = torch.jit.trace(model, dummy_inputs)
# torch.jit.save(traced_model, "../blip/model.pt")

<class 'dict'>
dict_keys(['vision_model.embeddings.class_embedding', 'vision_model.embeddings.position_embedding', 'vision_model.embeddings.patch_embedding.weight', 'vision_model.embeddings.patch_embedding.bias', 'vision_model.encoder.layers.0.self_attn.qkv.weight', 'vision_model.encoder.layers.0.self_attn.qkv.bias', 'vision_model.encoder.layers.0.self_attn.projection.weight', 'vision_model.encoder.layers.0.self_attn.projection.bias', 'vision_model.encoder.layers.0.layer_norm1.weight', 'vision_model.encoder.layers.0.layer_norm1.bias', 'vision_model.encoder.layers.0.mlp.fc1.weight', 'vision_model.encoder.layers.0.mlp.fc1.bias', 'vision_model.encoder.layers.0.mlp.fc2.weight', 'vision_model.encoder.layers.0.mlp.fc2.bias', 'vision_model.encoder.layers.0.layer_norm2.weight', 'vision_model.encoder.layers.0.layer_norm2.bias', 'vision_model.encoder.layers.1.self_attn.qkv.weight', 'vision_model.encoder.layers.1.self_attn.qkv.bias', 'vision_model.encoder.layers.1.self_attn.projection.weight', 'v